In [ ]:
# image net dataset - ISLVRC2012 local machine practise

In [1]:
class DefaultConfigs(object):

    train_dir = "../train"
    val_dir = '../val'
    model_name = "resnet18"
    weights = "./checkpoints/"
    best_models = weights + "best_model/"

    epochs = 40
    start_epoch = 0
    batch_size = 16
    momentum = 0.9
    lr = 1e-3
    weight_decay = 1e-4
    interval = 10
    workers = 12

    evaluate = False
    pretrained = False
    resume = False

In [2]:
class AverageMeter(object):

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


def accuracy(output, target, topk=(1,)):

    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [3]:
#validate model accuracy
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for batch_id, (images, target) in enumerate(val_loader):
            images, target = images.to(device), target.to(device)
            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if (batch_id + 1) % config.interval == 0:
                print('Acc@1: {top1.avg:.3f}\tAcc@5: {top5.avg:.3f}\tTime: {batch_time.val:.2f}\tID: {batch_id:d}'
                      .format(top1=top1, top5=top5, batch_time=batch_time, batch_id=(batch_id + 1) * config.batch_size))

        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    return top1.avg

In [4]:
def train(train_loader, model, criterion, optimizer):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')

    # switch to train mode
    model.train()

    end = time.time()
    for batch_id, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images, target = images.to(device), target.to(device)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if (batch_id + 1) % config.interval == 0:
            print('Acc@1: {top1.avg:.3f}\tAcc@5: {top5.avg:.3f}\t'
                  'Loss: {losses.val}\tTime: {batch_time.val:.2f}\tID: {batch_id:d}'
                  .format(top1=top1, top5=top5, batch_time=batch_time,
                          losses=losses, batch_id=(batch_id + 1) * config.batch_size))

    print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

In [ ]:
def main():
    global best_acc

    if config.pretrained:
        print("=> using pre-trained model '{}'".format(config.model_name))
        model = models.__dict__[config.model_name](pretrained=True)
    else:
        print("=> creating model '{}'".format(config.model_name))
        model = models.__dict__[config.model_name]()
    model.to(device)
